In [9]:
# Starting from loading the libraries required for analysis
# This was first done step by step with each model and cell but was combined finally to make the file cleaner and shorter  

!pip install pandas 
!pip install xgboost
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
import sklearn.preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [10]:
# Load dataset
file_path = 'Monthly_New.csv'
data = pd.read_csv(file_path)

# first few rows of the dataframe
data.head()

,Year,Month,TotalPrimaryEnergyConsumption,TotalPrimaryEnergyProduction,TotalPrimaryEnergyExports,TotalPrimaryEnergyImports,TotalPrimaryEnergyConsumedbytheIndustrialSector,TotalPrimaryEnergyConsumedbytheResidentialSector,PrimaryEnergyStockChange
0,1973,1,7223.87,5404.72,125.78,1173.08,2027.03,1339.27,771.858
1,1973,2,6592.37,5155.12,120.88,1168.01,1841.30,1174.90,390.129
2,1973,3,6521.44,5419.56,139.95,1309.47,1926.41,983.37,-67.640
3,1973,4,5941.73,5160.81,194.19,1085.17,1914.02,715.39,-110.067
4,1973,5,6071.94,5411.25,196.78,1162.80,2076.47,535.91,-305.335


In [11]:
# Creating a unique identifier for each year (sequence from 1 to 74) - To use it as a feature while building models
# Combining 'Year' and 'Month' columns into a new 'Date' column which displays time-series format
data['Date'] = pd.to_datetime(data['Year'].astype(str) + '-' + data['Month'].astype(str), format='%Y-%m')

data['t'] = range(1, len(data) + 1)

print(data.head())
data.info()



   Year  Month  TotalPrimaryEnergyConsumption  TotalPrimaryEnergyProduction  \
0  1973      1                        7223.87                       5404.72   
1  1973      2                        6592.37                       5155.12   
2  1973      3                        6521.44                       5419.56   
3  1973      4                        5941.73                       5160.81   
4  1973      5                        6071.94                       5411.25   

   TotalPrimaryEnergyExports  TotalPrimaryEnergyImports  \
0                     125.78                    1173.08   
1                     120.88                    1168.01   
2                     139.95                    1309.47   
3                     194.19                    1085.17   
4                     196.78                    1162.80   

   TotalPrimaryEnergyConsumedbytheIndustrialSector  \
0                                          2027.03   
1                                          1841.30   
2       

## LSTM Model Production

Clearly, LSTM model performed the best with lowest MAPE and RMSE. However, it is not accounting for the seasonality and is accounting for only trend. How to get the correct forecast graph. 

In [12]:
# Defining featues for production
features = ['TotalPrimaryEnergyConsumption', 'TotalPrimaryEnergyExports', 'TotalPrimaryEnergyImports', 't']
target = ['TotalPrimaryEnergyProduction']

# Initializing the MinMaxScaler
scaler = MinMaxScaler()

# Scaling the selected features
scaled_data = scaler.fit_transform(data[features])

# Converting the scaled data back to a dataframe for convenience
scaled_data_df = pd.DataFrame(scaled_data, columns=features, index=data.index)

# Splitting the data into a training set (1973-2016) and a test set (2017-2022)
train_data = scaled_data_df[(data['Year'] >= 1973) & (data['Year'] <= 2016)]
test_data = scaled_data_df[(data['Year'] >= 2017) & (data['Year'] <= 2022)]

# Reshaping the data to fit the LSTM input shape
X_train = train_data.values.reshape((train_data.shape[0], 1, train_data.shape[1]))
X_test = test_data.values.reshape((test_data.shape[0], 1, test_data.shape[1]))

# Since we're going to perform one-step-ahead prediction, the target is the same as the input shifted by one time step
y_train = train_data.values[1:]
y_test = test_data.values[1:]

# Trimming the last sample of the training and test input data since we don't have a label for it
X_train = X_train[:-1, :, :]
X_test = X_test[:-1, :, :]

# Defining the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model_lstm.add(Dense(y_train.shape[1]))

# Compiling the LSTM model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Training the LSTM model
history_lstm = model_lstm.fit(X_train, y_train, epochs=200, batch_size=42, validation_split=0.2, verbose=1)

# Evaluating the LSTM model on test data
test_loss_lstm = model_lstm.evaluate(X_test, y_test)

# Predictions
y_pred_lstm = model_lstm.predict(X_test)

# Mean Squared Error (MSE) for LSTM model
mse_lstm = mean_squared_error(y_test, y_pred_lstm)

# Mean Absolute Error (MAE) for LSTM model
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)

# Root Mean Squared Error (RMSE) for LSTM model
rmse_lstm = np.sqrt(mse_lstm)

# Print the results for LSTM model
print("LSTM Model Results for Production:")
print(f'Mean Squared Error (MSE): {mse_lstm:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse_lstm:.2f}')
print(f'Mean Absolute Error (MAE): {mae_lstm:.2f}')


Epoch 1/200
11/11 [==============================] - 2s 30ms/step - loss: 0.1591 - val_loss: 0.3515
Epoch 2/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1355 - val_loss: 0.2983
Epoch 3/200
11/11 [==============================] - 0s 6ms/step - loss: 0.1140 - val_loss: 0.2430
Epoch 4/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0913 - val_loss: 0.1852
Epoch 5/200
11/11 [==============================] - 0s 7ms/step - loss: 0.0690 - val_loss: 0.1274
Epoch 6/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0471 - val_loss: 0.0776
Epoch 7/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0301 - val_loss: 0.0444
Epoch 8/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0194 - val_loss: 0.0296
Epoch 9/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0137 - val_loss: 0.0259
Epoch 10/200
11/11 [==============================] - 0s 7ms/step - loss: 0.0118 - val_loss: 0.0281
Epoch 11

## LSTM Model Consumption

In [13]:
# Defining featues for consumption
features = ['TotalPrimaryEnergyProduction', 'TotalPrimaryEnergyExports', 'TotalPrimaryEnergyImports', 't', 'PrimaryEnergyStockChange']
target = ['TotalPrimaryEnergyConsumption']

# Initializing the MinMaxScaler
scaler = MinMaxScaler()

# Scaling the selected features
scaled_data = scaler.fit_transform(data[features])

# Converting the scaled data back to a dataframe for convenience
scaled_data_df = pd.DataFrame(scaled_data, columns=features, index=data.index)

# Splitting the data into a training set (1973-2016) and a test set (2017-2022)
train_data = scaled_data_df[(data['Year'] >= 1973) & (data['Year'] <= 2016)]
test_data = scaled_data_df[(data['Year'] >= 2017) & (data['Year'] <= 2022)]

# Reshaping the data to fit the LSTM input shape
X_train = train_data.values.reshape((train_data.shape[0], 1, train_data.shape[1]))
X_test = test_data.values.reshape((test_data.shape[0], 1, test_data.shape[1]))

# Since we're going to perform one-step-ahead prediction, the target is the same as the input shifted by one time step
y_train = train_data.values[1:]
y_test = test_data.values[1:]

# Trimming the last sample of the training and test input data since we don't have a label for it
X_train = X_train[:-1, :, :]
X_test = X_test[:-1, :, :]

# Defining the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model_lstm.add(Dense(y_train.shape[1]))

# Compiling the LSTM model
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Training the LSTM model
history_lstm = model_lstm.fit(X_train, y_train, epochs=200, batch_size=42, validation_split=0.2, verbose=1)

# Evaluating the LSTM model on test data
test_loss_lstm = model_lstm.evaluate(X_test, y_test)

# Predictions
y_pred_lstm = model_lstm.predict(X_test)


# Mean Squared Error (MSE) for LSTM model
mse_lstm = mean_squared_error(y_test, y_pred_lstm)

# Mean Absolute Error (MAE) for LSTM model
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)

# Root Mean Squared Error (RMSE) for LSTM model
rmse_lstm = np.sqrt(mse_lstm)

# Print the results for LSTM model
print("LSTM Model Results for Consumption:")
print(f'Mean Squared Error (MSE): {mse_lstm:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse_lstm:.2f}')
print(f'Mean Absolute Error (MAE): {mae_lstm:.2f}')


Epoch 1/200
11/11 [==============================] - 2s 25ms/step - loss: 0.1342 - val_loss: 0.2863
Epoch 2/200
11/11 [==============================] - 0s 8ms/step - loss: 0.1111 - val_loss: 0.2416
Epoch 3/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0872 - val_loss: 0.1882
Epoch 4/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0623 - val_loss: 0.1289
Epoch 5/200
11/11 [==============================] - 0s 5ms/step - loss: 0.0397 - val_loss: 0.0780
Epoch 6/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0245 - val_loss: 0.0500
Epoch 7/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0183 - val_loss: 0.0402
Epoch 8/200
11/11 [==============================] - 0s 7ms/step - loss: 0.0168 - val_loss: 0.0370
Epoch 9/200
11/11 [==============================] - 0s 8ms/step - loss: 0.0162 - val_loss: 0.0364
Epoch 10/200
11/11 [==============================] - 0s 6ms/step - loss: 0.0156 - val_loss: 0.0362
Epoch 11